# Use nn-Meter for Benchmark Dataset
In this notebook, we showed nn-Meter examples of latency prediction for benchmark dataset.

In [4]:
import nn_meter
from nn_meter.dataset import bench_dataset

datasets = bench_dataset()
for data in datasets:
    print(data)

/home/jiahang/.nn_meter/dataset/alexnets.jsonl
/home/jiahang/.nn_meter/dataset/densenets.jsonl
/home/jiahang/.nn_meter/dataset/googlenets.jsonl
/home/jiahang/.nn_meter/dataset/mnasnets.jsonl
/home/jiahang/.nn_meter/dataset/mobilenetv1s.jsonl
/home/jiahang/.nn_meter/dataset/mobilenetv2s.jsonl
/home/jiahang/.nn_meter/dataset/mobilenetv3s.jsonl
/home/jiahang/.nn_meter/dataset/nasbench201s.jsonl
/home/jiahang/.nn_meter/dataset/proxylessnass.jsonl
/home/jiahang/.nn_meter/dataset/resnets.jsonl
/home/jiahang/.nn_meter/dataset/resnets1.jsonl
/home/jiahang/.nn_meter/dataset/shufflenetv2s.jsonl
/home/jiahang/.nn_meter/dataset/squeezenets.jsonl
/home/jiahang/.nn_meter/dataset/vggs.jsonl


In [5]:
# list all supporting latency predictors
predictors = nn_meter.list_latency_predictors()
for p in predictors:
    print(f"[Predictor] {p['name']}: version={p['version']}")

[Predictor] cortexA76cpu_tflite21: version=1.0
[Predictor] adreno640gpu_tflite21: version=1.0
[Predictor] adreno630gpu_tflite21: version=1.0
[Predictor] myriadvpu_openvino2019r2: version=1.0


In [6]:
# specify basic predictor
predictor_name = 'adreno640gpu_tflite21' # user can change text here to test other predictors
predictor_version = 1.0
test_data = '/home/jiahang/.nn_meter/dataset/alexnets.jsonl'

import warnings
warnings.filterwarnings('ignore')

In [9]:
import jsonlines

# load predictor
predictor = nn_meter.load_latency_predictor(predictor_name, predictor_version)

# predict latency
with jsonlines.open(test_data) as data_reader:
    for i, item in enumerate(data_reader):
        graph = item["graph"]
        pred_lat = predictor.predict(graph, model_type="nnmeter-ir")
        real_lat = item[predictor_name]
        print(f'{test_data}[{i}]: predict: {pred_lat}, real: {real_lat}')
        

/home/jiahang/.nn_meter/dataset/alexnets.jsonl[0]: predict: 23.447085575244774, real: 24.4851
/home/jiahang/.nn_meter/dataset/alexnets.jsonl[1]: predict: 23.88567577635713, real: 23.9185
/home/jiahang/.nn_meter/dataset/alexnets.jsonl[2]: predict: 29.586297830632216, real: 30.3052
/home/jiahang/.nn_meter/dataset/alexnets.jsonl[3]: predict: 51.12333226388624, real: 52.089
/home/jiahang/.nn_meter/dataset/alexnets.jsonl[4]: predict: 4.937166470494069, real: 5.26442
/home/jiahang/.nn_meter/dataset/alexnets.jsonl[5]: predict: 14.996201148770357, real: 15.2265
/home/jiahang/.nn_meter/dataset/alexnets.jsonl[6]: predict: 9.262593840400982, real: 9.12046
/home/jiahang/.nn_meter/dataset/alexnets.jsonl[7]: predict: 13.91285961819858, real: 14.2242
/home/jiahang/.nn_meter/dataset/alexnets.jsonl[8]: predict: 15.022936121166751, real: 15.2457
/home/jiahang/.nn_meter/dataset/alexnets.jsonl[9]: predict: 12.443609556620194, real: 12.5989
/home/jiahang/.nn_meter/dataset/alexnets.jsonl[10]: predict: 15.97

In [ ]:
for filename in datasets:
    print(f'Start testing {filename} ...')
    True_lat = []
    Pred_lat = []
    index = 0
    with jsonlines.open(filename) as data_reader:
        for item in data_reader:
            graph = item["graph"]
            pred_lat = predictor.predict(graph, model_type="nnmeter-ir")
            real_lat = item[predictor_name]
            if real_lat != None:
                True_lat.append(real_lat)
                Pred_lat.append(pred_lat)
                index += 1
    if len(True_lat) > 0:
        rmse, rmspe, error, acc5, acc10, _ = nn_meter.latency_metrics(Pred_lat, True_lat)
        print(
            f'{filename} on {predictor_name}: rmse: {rmse}, 5%accuracy: {acc5}, 10%accuracy: {acc10}'
        )